In [1]:
!git clone  https://github.com/collaborative-earth/aicacia.git
%cd aicacia/notebooks

Cloning into 'aicacia'...
remote: Enumerating objects: 676, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 676 (delta 134), reused 196 (delta 94), pack-reused 401 (from 1)
Receiving objects: 100% (676/676), 3.58 MiB | 7.02 MiB/s, done.
Resolving deltas: 100% (273/273), done.
/content/aicacia/notebooks


In [ ]:
# Access scraped files
from google.colab import drive
drive.mount('/content/drive')
#!gdown https://drive.google.com/file/d/1LrXwPMgiok1zcn4i4LND7LOohslSmv8x/view?usp=drive_link -O /content/aicacia/functional_trait_literature_unsegmented_sentences.zip --fuzzy
#!unzip /content/aicacia/functional_trait_literature_unsegmented_sentences.zip

In [3]:
#INstall dependencies
%pip install datasets
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-embeddings-huggingface
%pip install llama-index-finetuning
%pip install llama-index
%pip install llama-index-llms-ollama
!pip install colab-xterm
%load_ext colabxterm
%pip install llama-index-vector-stores-chroma

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 

In [14]:
import json
import os
import sys
from pathlib import Path
sys.path.append("..")
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import Settings
# Generate embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from finetuning.src.node_parsers.tei_parser import TEINodeParser
from finetuning.src.node_parsers.node_filters import TEINodeFilter

In [12]:
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
chroma_client = chromadb.PersistentClient(
    path="test",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)
chroma_collection = chroma_client.create_collection("wri_markdown")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [7]:
markdown_path = "/content/drive/Shareddrives/Aicacia MicroLab/Scraped Data/WRI/Sample (first 3 pages)/Markdown"
# Flat reader read markdown as plain text: there is an option in llama index for markdown file importer, but the same
# job can be achieved with node parser
reader = SimpleDirectoryReader(input_dir=markdown_path,file_extractor = {".md" : FlatReader()})
docs = reader.load_data()

In [15]:
# This simply parse markdown using heading as delimiters
file_parser = MarkdownNodeParser()
# If we want to change chunk size
sentence_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
embed_model =HuggingFaceEmbedding(model_name="BAAI/bge-m3")
pipeline = IngestionPipeline(transformations=[file_parser, sentence_parser,embed_model],
                             vector_store= vector_store)
nodes = pipeline.run(documents=docs[:10],show_progress=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/1265 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2229 [00:00<?, ?it/s]

In [32]:
# Reload data for future use
db2 = chromadb.PersistentClient(path="test")
chroma_collection2 = db2.get_or_create_collection("wri_markdown")
vector_store2 = ChromaVectorStore(chroma_collection=chroma_collection)

In [1]:
chroma_collection2.get()['ids']

NameError: name 'chroma_collection2' is not defined